# OpenAI Function Calling In LangChain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")


In [54]:
from typing import List, Dict
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic and LangChain compatibility : https://python.langchain.com/v0.2/docs/how_to/pydantic_compatibility/ 

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [45]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [46]:
foo.name

'Joe'

In [48]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [49]:
foo.age

'bar'

In [50]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [51]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [52]:
foo_p.name

'Jane'

**Note**: The next cell is expected to fail.

In [53]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

In [84]:
class GetBlogArticle(BaseModel):
    """Call this to get information about a blog article, given its author's info"""
    author: pUser

In [89]:
class Classroom(BaseModel):
    students: List[pUser]

In [90]:
obj = Classroom(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [91]:
obj

Classroom(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [55]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [56]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function

In [17]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\luca5\AppData\Local\Temp\ipykernel_16220\1313296718.py:1: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [57]:
# new version
weather_function = convert_to_openai_function(WeatherSearch)

In [58]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [59]:
from langchain_openai import AzureChatOpenAI, ChatOpenAI

In [60]:
model = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0
                        )

In [61]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 68, 'total_tokens': 85, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-d80d8060-a5de-473f-b521-620bee302805-0', usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85})

In [62]:
model_with_function = model.bind(functions=[weather_function])

In [63]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 67, 'total_tokens': 84, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-73b9f625-e9d0-405f-8628-8afe48a0c385-0', usage_metadata={'input_tokens': 67, 'output_tokens': 17, 'total_tokens': 84})

## Forcing it to use a function

We can force the model to use a function

In [64]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [65]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 77, 'total_tokens': 84, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-eda5c2aa-3726-468c-a7c8-cb859cd794e7-0', usage_metadata={'input_tokens': 77, 'output_tokens': 7, 'total_tokens': 84})

In [66]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 72, 'total_tokens': 79, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-47904d92-8c11-403e-bed1-d9ab8f46207b-0', usage_metadata={'input_tokens': 72, 'output_tokens': 7, 'total_tokens': 79})

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [67]:
from langchain.prompts import ChatPromptTemplate

In [69]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [70]:
chain = prompt | model_with_function

In [71]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 73, 'total_tokens': 90, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-a56aa177-16cd-472e-a9c6-c5adf8d45611-0', usage_metadata={'input_tokens': 73, 'output_tokens': 17, 'total_tokens': 90})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [72]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [73]:
functions = [
    convert_to_openai_function(WeatherSearch),
    convert_to_openai_function(ArtistSearch),
]

In [75]:
model_with_functions = model.bind(functions=functions)

In [76]:
# attention à bien utiliser la nouvelle variable model_with_functions car .bind n'agit pas 'inplace'
model.invoke("what is the weather in sf?")

AIMessage(content="I can't provide real-time weather updates. For the current weather in San Francisco, you might want to check a reliable weather website or app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 14, 'total_tokens': 42, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-2be0e0c9-3028-4ce0-b788-5a0b5ec727e7-0', usage_metadata={'input_tokens': 14, 'output_tokens': 28, 'total_tokens': 42})

In [77]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 111, 'total_tokens': 128, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-ac0d8b09-63af-4df1-8d8f-ea261f6737cb-0', usage_metadata={'input_tokens': 111, 'output_tokens': 17, 'total_tokens': 128})

In [78]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 113, 'total_tokens': 134, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run-f7426e37-974a-41ff-80f2-71ed9c4bd11b-0', usage_metadata={'input_tokens': 113, 'output_tokens': 21, 'total_tokens': 134})

In [79]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 106, 'total_tokens': 116, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-d5be8270-5544-4f8c-bd91-bd8dfef0863b-0', usage_metadata={'input_tokens': 106, 'output_tokens': 10, 'total_tokens': 116})